In [1]:
%env MINIO_HOST=minio
%env MINIO_PORT=9000
%env AWS_SECRET_KEY=EI1RW90H9KpLAaG8AOJlK1rnsEnWhfM5gvcfqq9d
%env AWS_ACCESS_KEY=SFFTcTnl3QKhg5XxVW6O

env: MINIO_HOST=minio
env: MINIO_PORT=9000
env: AWS_SECRET_KEY=EI1RW90H9KpLAaG8AOJlK1rnsEnWhfM5gvcfqq9d
env: AWS_ACCESS_KEY=SFFTcTnl3QKhg5XxVW6O


In [2]:
import os
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,io.delta:delta-spark_2.12:3.2.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executorEnv.AWS_ACCESS_KEY", os.environ["AWS_ACCESS_KEY"]) \
    .config("spark.executorEnv.AWS_SECRET_KEY", os.environ["AWS_SECRET_KEY"]) \
    .config("spark.hadoop.fs.s3a.endpoint", f"http://{os.environ['MINIO_HOST']}:{os.environ['MINIO_PORT']}") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

In [7]:
input_path = "buffer/bluesky-803b3eb2/posts"
output_path = "landing/bluesky-803b3eb2/posts"

In [5]:
input_glob = f"s3a://{input_path}/*.jsonl"
df = spark.read.json(input_glob)

In [6]:
df.show()

+--------------------+--------------------+--------------------+--------------------+----------+-----------+-----------+------------+--------------------+--------------------+
|          author_did| author_display_name|       author_handle|          created_at|like_count|quote_count|reply_count|repost_count|                text|                 uri|
+--------------------+--------------------+--------------------+--------------------+----------+-----------+-----------+------------+--------------------+--------------------+
|did:plc:z7oora4xf...|            Creaftar|creaftar.bsky.social|2025-05-16T00:34:...|         0|          0|          0|           0|📅 Crossword love...|at://did:plc:z7oo...|
|did:plc:jco3xcyew...|     Wonders Norbits|wondersnorbet.bsk...|2025-05-16T00:33:...|         0|          0|          1|           0|Useful in artific...|at://did:plc:jco3...|
|did:plc:qg65q5ilh...|              Andrew|andrew-does-stuff...|2025-05-16T00:29:...|         1|          0|          0| 

In [8]:
df.write.format("delta").mode("append").save(f"s3a://{output_path}")

In [9]:
written_df = spark.read.format("delta").load(f"s3a://{output_path}")

In [10]:
written_df.show()

+--------------------+--------------------+--------------------+--------------------+----------+-----------+-----------+------------+--------------------+--------------------+
|          author_did| author_display_name|       author_handle|          created_at|like_count|quote_count|reply_count|repost_count|                text|                 uri|
+--------------------+--------------------+--------------------+--------------------+----------+-----------+-----------+------------+--------------------+--------------------+
|did:plc:is5eyhkd5...|                    |realaaai.bsky.social|2025-05-16T00:43:...|         0|          0|          0|           0|We are excited to...|at://did:plc:is5e...|
|did:plc:x6qiwrjsq...|                  VH|purplevh80.bsky.s...|2025-05-16T00:43:...|         0|          0|          0|           0|What is the end g...|at://did:plc:x6qi...|
|did:plc:zc7xnabun...|           MTimation|montanimation.bsk...|2025-05-16T00:38:52Z|         0|          0|          0|